### Zeichnungsgerät 2

Wir benutzen wieder den im Notebook [Automaten](Automaten.ipynb) beschriebenen Automat um einen Zeichnungsgerät zu steuern.  

Wir schreiben Python-Code, der Zeichungsanweisungen für unser Zeichnungsgerät generiert.
Als erstes wollen wir ein Schachbrett zeichnen. Das Schachbrett ist durch folgenden Dict beschrieben:
```python
chessboard_config = {
    'position': (20, 20),  # Position der linken oberen Ecke
    'width': 160,            
    'height': 160,
    'n': 2,                # Brett hat n x n Felder  
    'colors': ('grey', 'blue'),
}
```
**Beachte**: Ein Feld in Reihe n und Spalte m hat die Farbe `colors[0]` falls $m+n$ **gerade**,
sonst Farbe `colors[1]`.

Wir schreiben eine Funktion 
```python
def make_chessboard_cmd(position, width, height, n, colors):
    ...
```
welche die entsprechende Zeichnungsanweisung generiert.
Die Idee ist, dass wir diese Funktion mit
```python
make_chessboard_cmd(**chessboard_config)
```
aufrufen.

In [ ]:
n = 3
color_pattern = ''
for row in range(n):
    for col in range(n):
        color_idx = (row+col) % 2
        color_pattern = color_pattern + str(color_idx)
    color_pattern = color_pattern + '\n'

print(color_pattern)

In [ ]:
def get_color(col, row, colors):
    idx = (row+col) % 2
    return colors[idx]

In [ ]:
get_color(col=1, row=3, colors=('grey', 'blue'))

In [ ]:
chessboard_config = {
    'position': (20, 20),
    'width': 160,
    'height': 160,
    'n': 2,
    'colors': ('grey', 'blue')
}

In [ ]:
def make_chessboard_cmd(position, width, height, n, colors):
    cmds = []
    x0, y0 = position
    dx, dy = width/n, height/n

    cmds.append('u')  # hebe Stift von Blatt
    cmds.append(f'g{x0},{y0};')  # bewege Stift zur linken oberen Ecke
    cmds.append(f'fblack;l2;r{width},{height};')  # zeichen schwarzen Rahmen
    for row in range(n):
        for col in range(n):
            color = get_color(col, row, colors)
            cmds.append(f'f{color};')  # setze Fuellfarbe
            cmds.append(f'R{dx},{dy};')  # zeichne Feld
            cmds.append(f'G{dx},0;')  # bewege Stift eine Feldbreite nach rechts
        cmds.append(f'G{-width},{dy};')  # bewege Stift zur neuen Reihe
    return ''.join(cmds)

In [ ]:
cmd = make_chessboard_cmd(**chessboard_config)
cmd

In [ ]:
import zeichnungsautomat as ZA

WIDTH = 300
HEIGHT = 200


def read(cmd, drawingBoard=None):
    if drawingBoard is None:
        _, drawingBoard = ZA.get_automat_and_canvas(width=WIDTH, height=HEIGHT)
    drawingBoard.automaton.read(cmd)
    return drawingBoard

In [ ]:
chessboard_config['n'] = 3
chessboard_config['width'] = 200
read(make_chessboard_cmd(**chessboard_config))

### Spielsteine platzieren
Als nächstes möchten wir eine Funktion, die
eine Anweisung generiert, die einen Spielstein der Farbe `color` auf das Feld in 
Spalte `col` und
Reihe `row` 
setzt. Wir möchten die Funktion wie folgt aufrufen können.
```python
make_place_cmd(col, row, color, **chessboard_config, radius=0.8)
```
Ist Radius gleich 1, soll der Stein gerade auf dem Feld platz haben.

In [ ]:
def make_place_cmd(col, row, color, position, width, height, n, colors, radius=1):
    cmds = []
    x0, y0 = position
    dx, dy = width/n, height/n
    max_radius = min(dx, dy)/2 - 3
    radius = max_radius*radius

    cmds.append(f'uf{color};')  # hebe Stift und setze Fuellfarbe
    cmds.append(f'g{x0},{y0};')
    cmds.append(f'G{col*dx},{row*dy};')  # bewege Stift auf linke obere Ecke des Feldes
    cmds.append(f'G{dx/2},{dy/2};')  # bewege Stift zur Feldmitte
    cmds.append(f'C{radius};')

    return ''.join(cmds)

In [ ]:
make_place_cmd(2, 3, 'blue', **chessboard_config)

In [ ]:
db = read(make_chessboard_cmd(**chessboard_config))
db

In [ ]:
place_stone = make_place_cmd(0, 1, 'red', **chessboard_config, radius=0.5)
db = read(place_stone, db)

### Steine vom Brett entfernen
Weiter möchten wir eine Funktion, die
eine Anweisung generiert, die einen Spielstein vom Feld in 
Spalte `col` und
Reihe `row`
entfernt. Anstatt den Spielstein zu entfernen,
zeichnen wir einfach das entsprechende Spielfeld neu.

Wir möchten die Funktion wie folgt aufrufen können.
```python
make_clear_cmd(col, row, **chessboard_config)
```

In [ ]:
def make_clear_cmd(col, row, position, width, height, n, colors):
    cmds = []
    x0, y0 = position
    dx, dy = width/n, height/n
    x, y = x0+col*dx, y0+row*dy
    color = get_color(col, row, colors)

    cmds.append(f'uf{color};')
    cmds.append(f'g{x},{y};')
    cmds.append(f'R{dx},{dy};')

    return ''.join(cmds)

In [ ]:
make_clear_cmd(col, row, **chessboard_config)

In [ ]:
db = read(make_chessboard_cmd(**chessboard_config))
db

In [ ]:
place_stone1 = make_place_cmd(0, 1, 'red', **chessboard_config)
place_stone2 = make_place_cmd(1, 1, 'green', **chessboard_config)
remove_stone1 = make_clear_cmd(0, 1, **chessboard_config)
remove_stone2 = make_clear_cmd(1, 1, **chessboard_config)

In [ ]:
db = read(place_stone1, db)

In [ ]:
db = read(place_stone2, db)

In [ ]:
db = read(remove_stone1, db)

In [ ]:
db = read(remove_stone1, db)

In [ ]:
db = read(remove_stone2, db)